In [2]:
import json
import os
import nltk
import spacy
from operator import itemgetter
import itertools
import random
import Levenshtein
import difflib
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
def depickle(file):
    file = open(file, 'rb')
    data = pickle.load(file)
    file.close()
    return data

In [1]:
def similarity_between_rules(r, verbose =True):
    rule1 = r[0]
    rule2 = r[1]
    s1 = tag_words(rule1)
    s2 = tag_words(rule2)
    s = difflib.SequenceMatcher(None, s1, s2)
    verbose_list = []
    if verbose:
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            verbose_list.append((tag, s1[i1:i2], s2[j1:j2]))
#             print('{:7} {!r:>8} --> {!r}'.format(tag, s1[i1:i2], s2[j1:j2]))
    return (verbose_list,s.ratio())

def get_score_only(s1,s2,verbose=False):
    s = difflib.SequenceMatcher(None, s1, s2)
    verbose_list = []
    if verbose:
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            verbose_list.append((tag, s1[i1:i2], s2[j1:j2]))
#             print('{:7} {!r:>8} --> {!r}'.format(tag, s1[i1:i2], s2[j1:j2]))
    return (s.ratio(),verbose_list)
#     d = difflib.Differ()
#     result = list(d.compare(s1, s2))
#     if verbose:
#         from pprint import pprint
#         pprint(result)
#     return 
#     s = difflib.SequenceMatcher(None,s1,s2)
#     for opcode in s.get_opcodes():
#         print("%6s" s1[%d:%d], s2[%d:%d] % opcode)
#     print(Levenshtein.distance(s1,s2))
#     return Levenshtein.editops(s1,s2)
# [('replace', 1, 1), ('replace', 2, 2), ('replace', 3, 3)]

#     return s.get_opcodes()

def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 
    return list(itertools.combinations(arr, r))

def tag_words(s):
    nlp = spacy.load("en_core_web_sm")
    output_dir = 'my_mixed_random/'
    nlp2 = spacy.load(output_dir)
    ent_list = []
    doc2 = nlp2(s)
    for ent in doc2.ents:
        ent_list.append({'label':ent.label_, 'start':ent.start, 'end':ent.end})
    doc = nlp(s)
    for token in doc:
        ent_list.append({'label':token.pos_, 'start':token.idx, 'end': token.idx+len(token.text)})
    ent_list = sorted(ent_list,
                          key=lambda k: (k['start']))
    sign_list = []
    for tag in ent_list:
        sign_list.append(tag['label'])
    return sign_list
# r1= '(i)any person who is or has during the six months prior to the concerned act been associated with a company, directly or indirectly, in any capacity including by'
# r2 = 'reason of frequent communication with its officers or by being in any contractual, fiduciary or employment relationship or by being a director, officer or an employee of the company or holds any position including a professional or'
# r1 = 'Provided that in case the disclosure is made after twenty four hours of occurrence of the event or information, the listed entity shall, along with such disclosures provide explanation for delay:'
# r2 = 'Provided that in case the disclosure is made after twenty four hours of occurrence of the event or information, the listed entity shall, along with such disclosures provide explanation for the delay.'
r1 = '43.In order to remove any difficulties in respect of the application or interpretation of these regulations, the Board may issue clarifications or guidelines in the form of circulars.'
r2 = '27.In order to remove any difficulties in the interpretation or application of the provisions of these regulations, the Board may issue clarifications or guidelines from time to time.'
# print(r1)
# print(r2)
# similarity_between_rules((r1,r2))


In [3]:
rulewise_files = {}
list_rules_all =[]
file_array = os.listdir('data_rules/data')
for file_name in file_array:
    if file_name.endswith(".json"):
        f = open('data_rules/data/'+file_name)
        object_json2 = json.load(f)
        jj = list({v['id']:v for v in object_json2}.values())
        local_rules = []
        for rule in jj:
            if (not rule['rule'].isupper()) and('Page' not in rule['rule'])and('means' not in rule['rule']) and ('w.e.f.' not in rule['rule']) and ('_' not in rule['rule']) and ('SECURITIES AND EXCHANGE BOARD OF INDIA' not in rule['rule']):
                local_rules.append(rule['rule'])
                list_rules_all.append((rule['rule'], file_name))
        rulewise_files[file_name] = local_rules

In [4]:
new_rules = {}
for k in list_rules_all:
    try:
        if (not rule['rule'].isupper()) and('Page' not in rule['rule'])and('means' not in rule['rule']) and ('w.e.f.' not in rule['rule']) and ('_' not in rule['rule']) and ('SECURITIES AND EXCHANGE BOARD OF INDIA' not in rule['rule']) and (len(k[0])<=38) and (k[0][0]!='(') and (k[0][2]!='.')and (k[0][2]!=')')and (k[0][1]!='.'):
            pass
    except:
        pass

In [11]:
rules_with_tags = depickle('rule_tags_all_regulations_with_pos.pkl')

In [12]:
len(rules_with_tags)

7405

In [118]:
save_object(order,'similarity.pkl')

In [19]:
out = dict(list(rules_with_tags.items())[0: 20]) 

In [5]:
# out

In [15]:
new_rules = {}
count = 0
remove_list = ['12.(1) The Securities and Exchange Board of India (Prohibition of Insider Trading) Regulations, 1992 are hereby repealed.']
for rule in rules_with_tags:
    try:
        if count>200:
            break
#     if  ('w.e.f.' in rule):
#         k = rule.split('Inserted')
#         print(k)
        if ('No. LAD-NRO' not in rule)and(not rule.isupper()) and('Page' not in rule)and('means' not in rule) and ('w.e.f.' not in rule) and ('_' not in rule) and ('SECURITIES AND EXCHANGE BOARD OF INDIA' not in rule) and not((len(rule)<=38) and (rule[0]!='(') and (rule[2]!='.')and (rule[2]!=')')and (rule[1]!='.')):
    
#     if (('Page' not in rule) and ('w.e.f.' not in rule) and('means' not in rule)and(len(rule)>80)and(rule not in remove_list)):
                new_rules[rule] = [d for d in rules_with_tags[rule]]
#                 new_rules[rule] = [d['label'] for d in rules_with_tags[rule]]
                count += 1
    except:
        pass
print(count)

201


In [6]:
# new_rules

In [13]:
#Interfile rules' similarity
global_similarity= {}
# random.shuffle(new_rules)
# list_rules_all = list_rules_all[:5]
for rule1 in new_rules:
    for rule2 in new_rules:
        if rule1!= rule2:
            if (rule2,rule1) not in global_similarity.keys():
                global_similarity[(rule1,rule2)] =get_score_only(new_rules[rule1],new_rules[rule2])
# order = sorted(global_similarity.items(), key=itemgetter(1), reverse=True)
order = sorted(global_similarity.items(),key=lambda x: x[1][0], reverse=True)
# r_order = sorted(global_similarity.items(), key=itemgetter(1))


In [7]:
# import pickle
# # open a file, where you stored the pickled data
# file = open('similarity.pkl', 'rb')
# # dump information to that file
# order = pickle.load(file)
# # close the file
# file.close()
# data = []
# for rule in order:
# #     if (rule[0][0]!=rule[0][1]) and (rule[0][0]!=rule[0][1][[:-1]]):
#     if (rule[1]<1.0) and (rule[1]>=0.6):
#         if ('Inserted' not in rule[0]) and ('Substituted' not in rule[0]) and ('-' not in rule[0]):
#             print(rule[0][0])
#             print(rule[0][1])
#             print(rule[1])
#             data.append((rule[0][0], rule[0][1], rule[1]))
#     #     print(order[rule])
#             print('\n')

In [8]:
# data = []
# for rule in order:
# #     if (rule[0][0]!=rule[0][1]) and (rule[0][0]!=rule[0][1][[:-1]]):
#     if (rule[1]<1.0):
#         if ('Inserted' not in rule[0]) and ('Substituted' not in rule[0]) and ('-' not in rule[0]):
#             print(rule[0][0])
#             print(rule[0][1])
#             print(rule[1])
#             data.append((rule[0][0],rule[0][1],rule[1]))
#     #     print(order[rule])

#     print('\n')

In [9]:
# import csv
# csvfile = open('simi.csv','w', newline='')
# obj = csv.writer(csvfile)
# obj.writerows(data)
# obj.close()

In [10]:
# #Intrafile comparison of similarity of rules.
# #Storing intra rule score in a dictionary. and sort by value.
# same_file_similarty_ranking = {}
# for file in rulewise_files:
#     local_similarity = {}
#     rad = rSubset(rulewise_files[file], 2)
#     for r in rad:
# #         print(r[0])
# #         print(r[1])
# #         print(similarity_between_rules(r))
#         local_similarity[r] = similarity_between_rules(r)
# #         print('\n')

#     order = sorted(local_similarity.items(), key=itemgetter(1), reverse=True)
#     same_file_similarty_ranking[file] = order